In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
from torchvision import datasets

## Import data

In [2]:
cifar10_map = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck"
}
data_path = '../dataset/cifar10'
mean = [0.4914, 0.4822, 0.4465] # copied from Q1
std = [0.2470, 0.2435, 0.2616] # copied from Q1
batch_size = 25

In [3]:
# Load the CIFAR-10 dataset
cifar10_train = data.DataLoader(
    datasets.CIFAR10(
        root=data_path,
        train=True,
        download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])
    ),
    batch_size=batch_size,
    shuffle=True
)

Files already downloaded and verified


In [4]:
cifar10_test = data.DataLoader(
    datasets.CIFAR10(
        root=data_path,
        train=False,
        download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])
    ),
    batch_size=batch_size,
    shuffle=True
)

Files already downloaded and verified


# # Models For Convolutional Neural Network 

In [5]:
n_channel_in = 3
n_channel_1  = 16
n_channel_2  = 32
n_out = len(cifar10_map)

# Define the structure of the CNN
class ConvolutionalModel2(nn.Module):
    def __init__(self):
        super().__init__()
        # img is 32 x 32
        self.conv1 = nn.Conv2d(n_channel_in, n_channel_1, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        # img is 16 x 16
        self.conv2 = nn.Conv2d(n_channel_1, n_channel_2, kernel_size=3, padding=1)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)
        # img is 8 x 8
        
        # Add a fully connected layer at the end of the CNN
        self.fc_nn = nn.Linear(n_channel_2 * 8 * 8, n_out)

    def forward(self, x):
        # Apply the convolutional filters, activation functions, and pooling layers
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        
        # Flatten the output of the convolutional layers
        out = out.view(out.size(0), -1)
        # Apply the fully connected layer
        out = self.fc_nn(out)
        return out


In [6]:
n_channel_in = 3
n_channel_1  = 24
n_out = len(cifar10_map)

# Define the structure of the CNN
class ConvolutionalModel1(nn.Module):
    def __init__(self):
        super().__init__()
        # img is 32 x 32
        self.conv1 = nn.Conv2d(n_channel_in, n_channel_1, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        # img is 16 x 16
        
        # Add a fully connected layer at the end of the CNN
        self.fc = nn.Linear(n_channel_1 * 16 * 16, n_out)

    def forward(self, x):
        # Apply the convolutional filters, activation functions, and pooling layers
        out = self.pool1(self.act1(self.conv1(x)))
        
        # Flatten the output of the convolutional layers
        out = out.view(out.size(0), -1)
        # Apply the fully connected layer
        out = self.fc(out)
        return out


# Training function

In [7]:
def training(model, training_imgs, loss_fn, optimizer, n_epochs:int, report_period:int = 10):
    for epoch in range(n_epochs):
        for images, labels in training_imgs:
            # Zero the gradients
            optimizer.zero_grad()
            # Forward pass
            outputs = model(images)
            # Compute the loss
            loss = loss_fn(outputs, labels)
            # Backward pass
            loss.backward()
            # Update the parameters
            optimizer.step()
            
        if epoch % report_period == 0:
            print(f'epoch {epoch}:\t Loss: {loss:.15f}')

# 1 Layer Convolutional Neural Network
Question 2a

In [8]:
model = ConvolutionalModel1()
model 

ConvolutionalModel1(
  (conv1): Conv2d(3, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=6144, out_features=10, bias=True)
)

In [9]:
learn_rate = 1e-2
report_rate = 2
n_epochs = 300

optimizer = optim.Adam(model.parameters(), lr=learn_rate)

loss = nn.CrossEntropyLoss()

In [10]:
training(model, 
         cifar10_train, 
         loss, 
         optimizer, 
         n_epochs,
         report_period=report_rate
         )

[0]  Loss: 2.0867693424224854
[2]  Loss: 1.6567201614379883
[4]  Loss: 1.876031517982483
[6]  Loss: 2.1773462295532227
[8]  Loss: 2.1267964839935303
[10]  Loss: 1.554593563079834
[12]  Loss: 1.3991973400115967


KeyboardInterrupt: 

In [11]:
# Evaluate the CNN on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in cifar10_train:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Report the training time, training loss, and evaluation accuracy
print(f'Total: {total}')
print(f'Accuracy: {correct / total:.5f}')

Total: 50000
Accuracy: 0.467200


In [12]:
# Evaluate the CNN on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in cifar10_test:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Report the training time, training loss, and evaluation accuracy
print(f'Total: {total}')
print(f'Accuracy: {correct / total:.5f}')

Total: 10000
Accuracy: 0.388400


# 2 Layer Convolutional Neural Network
Question 2b

In [13]:
model = ConvolutionalModel2()
model 

ConvolutionalModel2(
  (conv1): Conv2d(3, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(24, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc_nn): Linear(in_features=2048, out_features=10, bias=True)
)

In [14]:
learn_rate = 1e-2
report_rate = 2
n_epochs = 300

optimizer = optim.Adam(model.parameters(), lr=learn_rate)

loss = nn.CrossEntropyLoss()

In [15]:
training(model, 
         cifar10_train, 
         loss, 
         optimizer, 
         n_epochs,
         report_period=report_rate
         )

[0]  Loss: 2.02242374420166
[2]  Loss: 1.7911823987960815
[4]  Loss: 1.6053640842437744
[6]  Loss: 1.82623291015625
[8]  Loss: 1.8039251565933228
[10]  Loss: 1.5589200258255005


KeyboardInterrupt: 

In [16]:
# Evaluate the CNN on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in cifar10_train:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Report the training time, training loss, and evaluation accuracy
print(f'Total: {total}')
print(f'Accuracy: {correct / total:.5f}')

Total: 50000
Accuracy: 0.348860


In [17]:
# Evaluate the CNN on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in cifar10_test:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Report the training time, training loss, and evaluation accuracy
print(f'Total: {total}')
print(f'Accuracy: {correct / total:.5f}')

Total: 10000
Accuracy: 0.340500
